# Prepare BW

In [ ]:
import brightway2 as bw
import os

## 1. Create bw project and set it to current 

In [ ]:
project = 'Geothermal'  
bw.projects.set_current(project)

## 2. Import biosphere and ecoinvent

In [ ]:
bw.bw2setup()

In [ ]:
ei_name = "ecoinvent 3.5 cutoff"    
ei_path = "/psi/home/kim_a/LCA files/ecoinvent 3.5 cutoff/datasets"
if ei_name in bw.databases:
    print(ei_name + " database already present!!! No import is needed")
else:
    ei = bw.SingleOutputEcospold2Importer(ei_path, ei_name)
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()

## 3. Import `geothermal energy` database

In [ ]:
bw.databases

In [ ]:
# %run Import_and_Replace.py

# Start DASK Client

In [ ]:
from dask.distributed import Client

In [ ]:
option_comp = 'local'

In [ ]:
if option_comp == "merlin":

    from dask_jobqueue import SLURMCluster
    
    cluster = SLURMCluster(cores     = 8, 
                           memory    ='4GB', 
                           walltime  = '06:00:00',
                           interface ='ib0',
                           local_directory = '/data/user/kim_a',
                           log_directory   = '/data/user/kim_a',
                           job_extra="--exclusive") 
    
elif option_comp == "local":
    
    from dask.distributed import LocalCluster
    
    cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
n_workers = 8
cluster.scale(n_workers+2)

In [ ]:
client

In [ ]:
# client.close()
# cluster.close()

# Project setup

In [ ]:
import numpy as np
import brightway2 as bw
from copy import copy

from utils.gsa_lca_dask import *
from setup_files_gsa import *

## --> TODO choose option: EGE or CGE

In [ ]:
option = 'cge'

## --> TODO choose number of Monte Carlo runs for one total index

In [ ]:
N = 60

# Create long task for each worker

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol

In [ ]:
def model_per_X_chunk(X_chunk, gsa_in_lca, method_matrices):
    scores = []
    i = 0
    for sample in X_chunk:
        score = gsa_in_lca.model(sample, method_matrices)
        scores.append(score)
        i += 1
    return np.array(scores)

In [ ]:
def task_per_worker(project, N, option, n_workers, i_chunk, path_files):

    # 1. setup geothermal project
    demand, methods, gt_model, parameters = setup_gt_project(project, option)

    # 4. generate characterization matrices for all methods
    lca = bw.LCA(demand, methods[0])
    lca.lci(factorize=True)
    lca.lcia()
    lca.build_demand_array()
    method_matrices = gen_cf_matrices(lca, methods)

    # 5. gsa in lca model
#     gsa_in_lca = GSAinLCA(lca, parameters, gt_model, options = ['geothermal energy'], project = project)
    gsa_in_lca = GSAinLCA(lca, parameters, gt_model, project = project)

    # 2. setup GSA project in the SALib format
    num_vars = len(gsa_in_lca.parameters_array) \
             + len(gsa_in_lca.uncertain_exchanges_dict['tech_params_where']) \
             + len(gsa_in_lca.uncertain_exchanges_dict['bio_params_where'])

    problem, calc_second_order = setup_gsa(num_vars)

    # 3. generate all samples, choose correct chunk for the current worker based on index i_chunk
    X = saltelli.sample(problem, N, calc_second_order=calc_second_order)
#     np.random.seed(10)
#     n_runs = N*(num_vars+2)
#     X = np.random.random([n_runs, num_vars])

    # Extract part of the sample for the current worker
    chunk_size = X.shape[0]//n_workers
    start = i_chunk*chunk_size
    if i_chunk != n_workers-1:
        end = (i_chunk+1)*chunk_size
    else:
        end = X.shape[0] 
    X_chunk = X[start:end, :]
    del X

    # 6. compute scores for all methods for X_chunk  
    scores_for_methods = model_per_X_chunk(X_chunk, gsa_in_lca, method_matrices)
    
    # 7. Save results
    filepath = os.path.join(path_files, 'scores_' + str(start) + '_' + str(end-1) + '.pkl')
    with open(filepath, "wb") as fp:   #Pickling
        pickle.dump(scores_for_methods, fp)

    return scores_for_methods

In [ ]:
# %%time
# i_chunk = 0
# scores_for_methods = task_per_worker(project, N, option, n_workers, i_chunk, path_files)

In [ ]:
# Path for saving results
path = "generated_files/write_files"
path_files = os.path.join(path, option + '_N' + str(N))
if not os.path.exists(path_files):
    os.makedirs(path_files)

# Compute with dask

In [ ]:
import dask

In [ ]:
task_per_worker = dask.delayed(task_per_worker)

In [ ]:
model_evals = []
for i in range(n_workers):
    model_eval = task_per_worker(project, N, option, n_workers, i, path_files)
    model_evals.append(model_eval)

In [ ]:
%%time
Y_intermediate = dask.compute(model_evals)

# Postprocessing of model outputs Y and saving

In [ ]:
Y_intermediate = np.array(Y_intermediate).squeeze()
Y_all_methods = np.vstack(Y_intermediate)

In [ ]:
Y_all_methods.shape

In [ ]:
filepath = os.path.join(path_files, 'all_scores.pkl')
with open(filepath, 'wb') as f:
    pickle.dump(Y_all_methods, f)

# Compute sobol indices

In [ ]:
import brightway2 as bw
import numpy as np
from SALib.analyze import sobol
import pickle, json, os

from setup_files_gsa import *
from utils.gsa_lca_dask import *

In [ ]:
option = 'cge'

In [ ]:
if option == 'cge':
    path = 'generated_files/write_files/cge_N60'
elif option == 'ege':
    path = 'generated_files/write_files/ege_N60'

In [ ]:
scores = get_lcia_results(path)

In [ ]:
scores.shape

In [ ]:
demand, methods, gt_model, parameters = setup_gt_project(project, option)

lca = bw.LCA(demand, methods[0])
lca.lci()
lca.lcia()
lca.build_demand_array()
gsa_in_lca = GSAinLCA(lca, parameters, gt_model, project = project)

num_vars = len(gsa_in_lca.parameters_array) \
         + len(gsa_in_lca.uncertain_exchanges_dict['tech_params_where']) \
         + len(gsa_in_lca.uncertain_exchanges_dict['bio_params_where'])
problem, calc_second_order = setup_gsa(num_vars)

sa_dict = {}
i = 0
sa_dict['parameters'] = gsa_in_lca.parameters_array['name'].tolist()
for Y in scores.transpose():
    method_name = methods[i][-1]
    dict_ = sobol.analyze(problem, Y, print_to_console=False, calc_second_order=calc_second_order)
    for k,v in dict_.items():
        dict_[k] = v.tolist()
    sa_dict[method_name] = dict_
    i += 1

In [ ]:
# Save Sobol indices
path_sob = os.path.join(path, 'sobol_indices' + '.json')
with open(path_sob, 'w') as f:
    json.dump(sa_dict, f)

# SA indices convergence

In [1]:
import brightway2 as bw
import numpy as np
from SALib.analyze import sobol
import pickle, json, os

from setup_files_gsa import *

In [2]:
# TODO --> choose option and for which parameter to plot SA convergence which_input = 12
option = 'cge'

In [3]:
if option == 'cge':
    path = 'generated_files/write_files/cge_N60'
elif option == 'ege':
    path = 'generated_files/write_files/ege_N60'

In [4]:
path_sob = os.path.join(path, 'sobol_indices' + '.json')
with open(path_sob, 'r') as f:
    sa_dict = json.load(f)
    
inputs = sa_dict['parameters']
D = len(inputs)

In [5]:
prob, cso = setup_gsa(D)

project = 'Geothermal'
_, methods, _, _ = setup_gt_project(project, option)
n_methods = len(methods)

## Plot error convergence

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import colorlover as cl
from IPython.display import HTML

In [7]:
colors = cl.scales['3']['qual']['Dark2']
HTML(cl.to_html( colors ))

In [8]:
scores = get_lcia_results(path)

N_start = 1
N_end = 60
N_step = 1

Ns = np.arange(N_start, N_end, N_step)
Ns = Ns[:-1]
first_d = np.zeros([Ns.shape[0], n_methods, len(inputs)])
total_d = np.zeros([Ns.shape[0], n_methods, len(inputs)])

i = 0
for m, method in enumerate(methods):
    i = 0
    for n in Ns:
        num_runs = n * (D+2)
        scores_n = scores[:num_runs, m]
        sa_dict = sobol.analyze(prob, scores_n, calc_second_order=cso)
        first = sa_dict['S1']
        total = sa_dict['ST']
        first_d[i, m, :] = first
        total_d[i, m, :] = total
        i += 1

### Plot all methods for one parameter

In [16]:
# TODO --> for which parameter to plot SA convergence 
which_input = 3

In [17]:
methods_names = [m[-1] for m in methods]

In [23]:
cols = 2

rows = int(np.ceil(len(methods)/cols))

fig = make_subplots(rows=rows, 
                    cols=cols, 
                    start_cell="top-left",
                    subplot_titles=methods_names
                   )

showleg = True
for m, method in enumerate(methods):
    fig.add_trace(go.Scatter(x=Ns, 
                             y=first_d[:, m, which_input ],
                             mode='lines+markers',
                             line=go.scatter.Line(color=colors[0]),
                             name = 'First index',
                             showlegend = showleg, 
                             ),
                  row=m//cols+1, 
                  col=m%cols+1, 
                  secondary_y=False,
                  )
     
    fig.add_trace(go.Scatter(x=Ns, 
                             y=total_d[:, m, which_input],
                             mode='lines+markers',
                             line=go.scatter.Line(color=colors[2]),
                             name = 'Total index',
                             showlegend = showleg),
                  row=m//cols+1, 
                  col=m%cols+1, 
                  secondary_y=False)

    showleg = False
    
    
fig.update_layout(width=990,
                  height = 1100, 
                  legend_orientation="h",
                  legend=dict(x=0.67, y=1.1, font_size=16),
                  title = inputs[which_input].replace('_',' ').capitalize(),
                 )

max_y = max(max(first_d[:, :, which_input].flatten()), max(total_d[:, :, which_input].flatten()))
min_y = min(min(first_d[:, :, which_input].flatten()), min(total_d[:, :, which_input].flatten()))

fig.update_yaxes(range=[min_y-0.5, max_y+0.5])

fig.show()

### Plot for all parameters for one method

In [13]:
which_method = 5

In [14]:
inputs_names = [inp.replace('_', ' ') for inp in inputs]

In [20]:
cols = 2
rows = int(np.ceil(len(inputs)/cols))

fig = make_subplots(rows=rows, 
                    cols=cols, 
                    start_cell="top-left",
                    subplot_titles=inputs_names
                   )

showleg = True
for i, inp in enumerate(inputs):
    fig.add_trace(go.Scatter(x=Ns, 
                             y=first_d[:, which_method, i],
                             mode='lines+markers',
                             line=go.scatter.Line(color=colors[0]),
                             name = 'First index',
                             showlegend = showleg, 
                             ),
                  row=i//cols+1, 
                  col=i%cols+1, 
                  secondary_y=False,
                  )
     
    fig.add_trace(go.Scatter(x=Ns, 
                             y=total_d[:, which_method, i],
                             mode='lines+markers',
                             line=go.scatter.Line(color=colors[2]),
                             name = 'Total index',
                             showlegend = showleg),
                  row=i//cols+1, 
                  col=i%cols+1, 
                  secondary_y=False)

    showleg = False
    
    
fig.update_layout(width=990,
                  height = 1100, 
                  legend_orientation="h",
                  legend=dict(x=0.67, y=1.1, font_size=16),
                  title = methods_names[which_method].capitalize(),
                 )

max_y = max(max(first_d[:, which_method, :].flatten()), max(total_d[:, which_method, :].flatten()))
min_y = min(min(first_d[:, which_method, :].flatten()), min(total_d[:, which_method, :].flatten()))

fig.update_yaxes(range=[min_y-0.5, max_y+0.5])

fig.show()

# ----------------------------------------THE END----------------------------------------

# Fix zero total indices

In [ ]:
from utils.gsa_lca_dask import *

In [ ]:
demand, methods, gt_model, parameters = setup_gt_project(project, option)

# 4. generate characterization matrices for all methods
lca = bw.LCA(demand, methods[0])
lca.lci(factorize=True)
lca.lcia()
lca.build_demand_array()
method_matrices = gen_cf_matrices(lca, methods)

gsa_in_lca = GSAinLCA(lca, parameters, gt_model, project = project)

In [ ]:
from pypardiso import spsolve
from copy import copy, deepcopy

In [ ]:
def compute_lsa_scores(where, kind, lca, CF_mats):
    
    scores_dict = {}
    
    where_len = where.shape[0]
    if where_len == 0:
        return scores_dict
    
    N = 2
    const_factor = (2-0.5)*np.random.random((where_len, N)) + 0.5
    
    if kind == 'tech':
        rows = lca.tech_params[where]['row']
        cols = lca.tech_params[where]['col']
    elif kind == 'bio':
        rows = lca.bio_params[where]['row']
        cols = lca.bio_params[where]['col']
    
    for i in range(where_len):
        
        scores = np.zeros((len(CF_mats), N))
        
        for j in range(N):
            A = deepcopy(lca.technosphere_matrix)
            B = deepcopy(lca.biosphere_matrix)
            x = deepcopy(spsolve(A, lca.demand_array))
            
            if kind == 'tech':
                A[rows[i], cols[i]] *= const_factor[i,j]
                x = spsolve(A, lca.demand_array)
            elif kind == 'bio':
                B[rows[i], cols[i]] *= const_factor[i,j]
                
            for k, CF_vec in enumerate(CF_mats):
                scores[k,j] = CF_vec * B * x
                
            del A, B, x
                
        scores_dict[int(where[i])] = copy(scores)
        
    return scores_dict

In [ ]:
uncertain_exchanges_dict = gsa_in_lca.uncertain_exchanges_dict

def local_sa(uncertain_exchanges_dict, lca): 
    
    N = 2
    B = lca.biosphere_matrix
    lca.build_demand_array()
    
    CF_mats = []
    for mat in method_matrices:
        CF_mats.append(sum(mat))
    
    # 1. Technosphere
    scores_dict_tech = compute_lsa_scores(uncertain_exchanges_dict['tech_params_where'], 
                                          'tech', 
                                          lca, 
                                          CF_mats)
    # 2. Biosphere
    scores_dict_bio  = compute_lsa_scores(uncertain_exchanges_dict['bio_params_where'], 
                                          'bio', 
                                          lca, 
                                          CF_mats)
    
    return scores_dict_tech, scores_dict_bio

In [ ]:
%%time
scores_dict_tech, scores_dict_bio = local_sa(gsa_in_lca.parameterized_exchanges_dict, lca)

In [ ]:
import json

In [ ]:
path = 'generated_files/gsa_results/cge_N560_LT_uniform.json'

In [ ]:
with open(path, 'rb') as f:
    a = json.load(f)

In [ ]:
for i in range
s1_cc = a['climate change total']['S1']
s1_ce = a['carcinogenic effects']['S1']

In [ ]:
np.where([s1_cc[i] == 0 for i in range(len(s1_ce))])

In [ ]:
np.where([s1_ce[i] == 0 for i in range(len(s1_ce))])

In [ ]:
list(a.keys())

# Draft

In [ ]:
model_evals

In [ ]:
%%time
Y = dask.compute(model_evals)

In [ ]:
# Sobol indices
model = dask.delayed(model)
model_evals = []
for sample in X:
    model_eval = model(sample)
    model_evals.append(model_eval)

In [ ]:
%%time
n_batches = len(model_evals) // n_workers
start = 0
Y = [0]*n_batches
for i_batch in range(n_batches):
    end = (i_batch+1)*n_workers
    Y[i_batch] = dask.compute(model_evals[start:end])
    start = end

In [ ]:
Y

In [ ]:
Y1 = np.array(Y).flatten()
Y1

In [ ]:
sa_sobol = sobol.analyze(problem, Y1, print_to_console=False, calc_second_order=calc_second_order)

In [ ]:
sa_sobol

# Run for all methods and store resutls

## 1. Dask

In [ ]:
from dask.distributed import Client
# from dask.distributed import LocalCluster
from dask_jobqueue import SLURMCluster

In [ ]:
cluster = SLURMCluster(cores     = 8, 
                       memory    ='4GB', 
                       walltime  = '02:00:00',
                       interface ='ib0',
                       local_directory = '/data/user/kim_a',
                       log_directory   = '/data/user/kim_a')
# if __name__ == '__main__':
#     cluster = SLURMCluster(cores = 8, memory = '1GB')
# cluster = LocalCluster(local_directory = '/data/user/kim_a')

In [ ]:
client = Client(cluster)

In [ ]:
n_workers = 40
cluster.scale(n_workers)

In [ ]:
client

In [ ]:
!hostname

## 2. Project setup

In [ ]:
import brightway2 as bw
import numpy as np
import pandas as pd
from copy import copy

from gsa_lca_dask import *

In [ ]:
#Set current project
project = 'Geothermal'
bw.projects.set_current(project)

#Local files
from lookup_func import lookup_geothermal
from cge_model import GeothermalConventionalModel
from ege_model import GeothermalEnhancedModel

#Choose demand/
_, _, _, _, _, _, _, _, _, _, _, _, _, _, electricity_prod_conv, electricity_prod_enha = lookup_geothermal()

#Choose LCIA method
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

In [ ]:
option = 'cge'
if option == 'cge':
    demand = {electricity_prod_conv: 1}
    from cge_klausen import parameters
    GTModel = GeothermalConventionalModel
elif option == 'ege':
    demand = {electricity_prod_enha: 1}
    from ege_klausen import parameters
    GTModel = GeothermalEnhancedModel

In [ ]:
N = 40

n_dimensions = len(parameters.data)
calc_second_order = False
problem = {
  'num_vars': n_dimensions,
  'names':    np.arange(n_dimensions),
  'bounds':   np.array([[0,1]]*n_dimensions)
}

## 3. dask.delayed

In [ ]:
import dask

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol
calc_second_order = False
X = saltelli.sample(problem, N, calc_second_order=calc_second_order)

def get_model_evals(model, X):
    model_evals = []
    for sample in X:
        model_eval = model(sample)
        model_evals.append(model_eval)        
    return model_evals


def get_Y(model_evals):
    n_batches = len(model_evals) // n_workers
    start = 0
    Y = [0]*n_batches
    for i_batch in range(n_batches):
        end = (i_batch+1)*n_workers
        Y[i_batch] = dask.compute(model_evals[start:end])
        start = end
    return Y
        

def sobol_wrt_method(demand, method, option):

    lca = bw.LCA(demand, method)
    lca.lci()
    lca.lcia()
    lca.build_demand_array()

    print(method)

    def model(sample):
        if option == 'cge':
            from cge_klausen import parameters
            GTModel = GeothermalConventionalModel
        elif option == 'ege':
            from cge_klausen import parameters
            GTModel = GeothermalEnhancedModel
        lca_ = copy(lca)
        gt_model = GTModel(parameters)
        gsa_in_lca = GSAinLCA(project, demand, method, parameters, gt_model, lca_)
        return gsa_in_lca.model(sample)
    
    model = dask.delayed(model)
    
    model_evals = get_model_evals(model, X)
    Y = get_Y(model_evals)
    Y = np.array(Y).flatten()
    sa_sobol = sobol.analyze(problem, Y, print_to_console=False, calc_second_order=calc_second_order)
    
    return sa_sobol

In [ ]:
import os
import json

In [ ]:
sa_sobol = {}
path = "/psi/home/kim_a/geothermal/write_files" + '_' + option
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
#save names of parameters
method = ILCD[0]
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
    
path_n = os.path.join(path, 'parameters.json')
gt_model = GTModel(parameters)
gsa_in_lca = GSAinLCA(project, demand, method, parameters, gt_model, lca)

with open(path_n, 'w') as f:
    json.dump(gsa_in_lca.parameters_array['name'].tolist(), f)

In [ ]:
import time

In [ ]:
%%time
for method in ILCD:
    method_name = method[-1]
    
    t0 = time.time()
    sa_dict = sobol_wrt_method(demand, method, option)
    t1 = time.time()
    print('Time needed for this method: ' + str(t1-t0))
    
    sa_sobol[method_name] = sa_dict
    
    # Save
    for k,v in sa_dict.items():
        sa_dict[k] = v.tolist()
    path_m = os.path.join(path, method[-1].replace(' ', '_') + '.json')
    with open(path_m, 'w') as f:
        json.dump(sa_dict, f)

In [ ]:
import numpy as np
import sys

In [ ]:
a = np.random.random(2077)

In [ ]:
sys.getsizeof(a)

# Playground

In [ ]:
from gsa_lca_dask import GSAinLCA
from cge_klausen import parameters
from cge_model import GeothermalConventionalModel
from lookup_func import lookup_geothermal

In [ ]:
bw.projects.set_current('Geothermal')

In [ ]:
from gsa_lca_dask import GSAinLCA
from cge_klausen import parameters
from cge_model import GeothermalConventionalModel
from lookup_func import lookup_geothermal

In [ ]:
_, _, _, _, _, _, _, _, _, _, _, _, _, _, electricity_prod_conv, electricity_prod_enha = lookup_geothermal()
demand = {electricity_prod_conv: 1}
method = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) 
          and "climate change total" in str(method)][0]
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
print(lca.score)

# Close Client

In [ ]:
client.close()